In [65]:
include("../../src/WaterLily.jl")

using Plots
using Statistics

In [66]:
optPois = "Pois"
optSour = "HalfPerEasy"
optBCon = "Per"

stepbystep = false

computationID = optPois * "_" * optSour * "_" * optBCon * "_"
print(computationID,"\n")

Pois_HalfPerEasy_Per_


In [67]:
T=Float32
N = (2^7,3*2^7)
D = length(N)
Ng = N .+ 2
Nd = (Ng..., 2)

offset = -0.3


optSour=="Hard" && (SourceFunc(x,y) = (
    -4 *pi^2 * (1/N[1]^2 + 1/N[2]^2) * sin.(2*pi*(x/N[1].+offset)) .* sin.(2*pi*(y/N[2].+offset)) +
    -16*pi^2 * (1/N[1]^2 + 1/N[2]^2) * sin.(8*pi*(x/N[1].+offset)) .* sin.(8*pi*(y/N[2].+offset))
))

optSour=="Hard" && (ManufacSol(x,y) = (
    sin.(2*pi*(x/N[1].+offset)) .* sin.(2*pi*(y/N[2].+offset)) +
    sin.(8*pi*(x/N[1].+offset)) .* sin.(8*pi*(y/N[2].+offset))/4
))

optSour=="HardHalPer" && (SourceFunc(x,y) = (
    -4 *pi^2 * (1/N[1]^2 + 1/N[2]^2) * cos.(2*pi*(x/N[1])) .* cos.(2*pi*(y/N[2].+offset)) +
    -16*pi^2 * (1/N[1]^2 + 1/N[2]^2) * cos.(8*pi*(x/N[1])) .* cos.(8*pi*(y/N[2].+offset))
))

optSour=="HardHalPer" && (ManufacSol(x,y) = (
    cos.(2*pi*(x/N[1])) .* cos.(2*pi*(y/N[2].+offset)) +
    cos.(8*pi*(x/N[1])) .* cos.(8*pi*(y/N[2].+offset))/4
))

optSour=="Easy" && (SourceFunc(x,y) = (
    4*pi^2 * (
        cos.(2*pi*(x/N[1])) .* (1 .-cos.(2*pi*(y/N[2])))/N[1]^2 +
        (1 .-cos.(2*pi*(x/N[1]))) .* cos.(2*pi*(y/N[2]))/N[2]^2
    )
))

optSour=="Easy" && (ManufacSol(x,y) = (
    (1 .-cos.(2*pi*(x/N[1]))) .* (1 .-cos.(2*pi*(y/N[2]))) 
))


waveNum = 4*pi
optSour=="Middle" && (SourceFunc(x,y) = (
    waveNum^2 * (
        cos.(waveNum*(x/N[1])) .* (1 .-cos.(waveNum*(y/N[2])))/N[1]^2 +
        (1 .-cos.(waveNum*(x/N[1]))) .* cos.(waveNum*(y/N[2]))/N[2]^2
    )
))

optSour=="Middle" && (ManufacSol(x,y) = (
    (1 .-cos.(waveNum*(x/N[1]))) .* (1 .-cos.(waveNum*(y/N[2]))) 
))

optSour=="HalfPerEasy" && (SourceFunc(x,y) = (
    -pi^2*(1/N[1]^2 + 4/N[2]^2) * cos.(1*pi*(x/N[1])) .* sin.(2*pi*(y/N[2]))
))

optSour=="HalfPerEasy" && (ManufacSol(x,y) = (
    cos.(1*pi*(x/N[1])) .* sin.(2*pi*(y/N[2]))
))

ManufacSol (generic function with 1 method)

In [68]:
solIni = rand(Ng...)*10
IniOrig = copy(solIni)
source = zeros(Ng)
μ₀ = ones(Nd)
# WaterLily.BCVecPerNeu!(μ₀,Dirichlet=true, A=zeros(2),perdir=(2,))
#optPois=="MulP" && WaterLily.BC!(μ₀,ntuple(zero, D))

optBCon == "Neu" && WaterLily.BC!(solIni)
optBCon == "Per" && WaterLily.BCPer!(solIni)
WaterLily.BCPerNeu!(solIni)
# BCPer!(source)
# BCPerVec!(μ₀)

X = transpose(reshape([i-1.5 for i=1:Ng[1] for j=1:Ng[2]],Ng[2],Ng[1]))
Y = transpose(reshape([j-1.5 for i=1:Ng[1] for j=1:Ng[2]],Ng[2],Ng[1]))


source = SourceFunc(X,Y)
realSol = ManufacSol(X,Y)
originalSource = copy(source)

realSol .-= mean(realSol[2:end-1,2:end-1])

130×386 Matrix{Float64}:
 -0.00818052   0.00818052   0.0245394  …  -0.00818052   0.00818052
 -0.00818052   0.00818052   0.0245394     -0.00818052   0.00818052
 -0.0081756    0.0081756    0.0245246     -0.0081756    0.0081756
 -0.00816574   0.00816574   0.024495      -0.00816574   0.00816574
 -0.00815097   0.00815097   0.0244507     -0.00815097   0.00815097
 -0.00813129   0.00813129   0.0243917  …  -0.00813129   0.00813129
 -0.00810671   0.00810671   0.024318      -0.00810671   0.00810671
 -0.00807725   0.00807725   0.0242296     -0.00807725   0.00807725
 -0.00804292   0.00804292   0.0241266     -0.00804292   0.00804292
 -0.00800375   0.00800375   0.0240091     -0.00800375   0.00800375
  ⋮                                    ⋱                ⋮
  0.00804292  -0.00804292  -0.0241266      0.00804292  -0.00804292
  0.00807725  -0.00807725  -0.0242296      0.00807725  -0.00807725
  0.00810671  -0.00810671  -0.024318       0.00810671  -0.00810671
  0.00813129  -0.00813129  -0.0243917      0.00

In [69]:
optPois=="Pois" && (pTest = WaterLily.Poisson(solIni, μ₀, source,perdir=(2,)))
optPois=="MulP" && (pTest = WaterLily.MultiLevelPoisson(solIni, μ₀, source,perdir=(2,)))


false

In [70]:
res = []
stepbystep && for i ∈ 1:200
    partRes = WaterLily.solver!(pTest;log=true,tol=1e-12,itmx=20)
    # pTest.x .-= mean(pTest.x[2:end-1,2:end-1]) 
    pythonplot()
    contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],pTest.x[2:end-1,2:end-1];levels=32)
    savefig("bunchFig/" * computationID * "Calculated_"*string(i, pad=4)*".png")
    # pythonplot()
    # contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(realSol[2:end-1,2:end-1]-pTest.x[2:end-1,2:end-1]).+1e-12);levels=32)
    # savefig("bunchFig/" * computationID * "Error_"*string(i, pad=4)*".png")

    push!(res,partRes...)
end

!stepbystep && (res = WaterLily.solver!(pTest;log=true,tol=1e-12,itmx=5e3));

#print(res)

5001-element Vector{Float64}:
    8.128475641730766e6
 1633.5937555234173
   72.25817376427159
   24.46363552056929
    6.656662233214044
    4.28182699809887
    1.748724268033535
    1.4534453870372093
    0.712762283920526
    0.6784689310340324
    ⋮
    8.569184651961522e-9
    8.709424686978414e-9
    8.566878199475934e-9
    8.707080491595573e-9
    8.56457237492406e-9
    8.704736934418871e-9
    8.562267178128868e-9
    8.702394015268375e-9
    8.559962608913316e-9

In [71]:
pTest.x .-= mean(pTest.x[2:end-1,2:end-1]) 
realSol .-= mean(realSol[2:end-1,2:end-1]) 

130×386 Matrix{Float64}:
 -0.00818052   0.00818052   0.0245394  …  -0.00818052   0.00818052
 -0.00818052   0.00818052   0.0245394     -0.00818052   0.00818052
 -0.0081756    0.0081756    0.0245246     -0.0081756    0.0081756
 -0.00816574   0.00816574   0.024495      -0.00816574   0.00816574
 -0.00815097   0.00815097   0.0244507     -0.00815097   0.00815097
 -0.00813129   0.00813129   0.0243917  …  -0.00813129   0.00813129
 -0.00810671   0.00810671   0.024318      -0.00810671   0.00810671
 -0.00807725   0.00807725   0.0242296     -0.00807725   0.00807725
 -0.00804292   0.00804292   0.0241266     -0.00804292   0.00804292
 -0.00800375   0.00800375   0.0240091     -0.00800375   0.00800375
  ⋮                                    ⋱                ⋮
  0.00804292  -0.00804292  -0.0241266      0.00804292  -0.00804292
  0.00807725  -0.00807725  -0.0242296      0.00807725  -0.00807725
  0.00810671  -0.00810671  -0.024318       0.00810671  -0.00810671
  0.00813129  -0.00813129  -0.0243917      0.00

In [72]:
pythonplot()
plot(res)
plot!(yscale=:log10, minorgrid=true)
savefig(computationID * "Residual.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_HalfPerEasy_Per_Residual.png"

In [73]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],pTest.x[2:end-1,2:end-1];levels=32)
savefig(computationID * "Calculated.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_HalfPerEasy_Per_Calculated.png"

In [74]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(realSol[2:end-1,2:end-1]-pTest.x[2:end-1,2:end-1]).+1e-12);levels=32)
savefig(computationID * "Error.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_HalfPerEasy_Per_Error.png"

In [75]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],realSol[2:end-1,2:end-1];levels=32)
savefig(computationID * "Correct.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_HalfPerEasy_Per_Correct.png"

In [76]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],originalSource[2:end-1,2:end-1];levels=32)
savefig(computationID * "Source.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_HalfPerEasy_Per_Source.png"

In [77]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],IniOrig[2:end-1,2:end-1];levels=32)
savefig(computationID * "InitialSol.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_HalfPerEasy_Per_InitialSol.png"